In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from anchor.visualize import MODALITY_TO_COLOR, MODALITY_ORDER, MODALITY_PALETTE
modality_order = MODALITY_ORDER

sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline

%load_ext autoreload
%autoreload 2

folder = 'figures'

import flotilla
# study = flotilla.embark('singlecell_pnm_figure2_modalities_bayesian_kmers_cisbp', 
study = flotilla.embark('singlecell_pnm_figure2_modalities_rmdup', 
                        flotilla_dir='/projects/ps-yeolab/obotvinnik/flotilla_projects/')
# study = flotilla.embark('singlecell_pnm_figure1_supplementary_post_splicing_filtering')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2017-01-05 15:54:17	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities_rmdup/datapackage.json
2017-01-05 15:54:17	Parsing datapackage to create a Study object
https://s3-us-west-2.amazonaws.com/flotilla-projects/ercc/ERCC_Controls.txt has not been downloaded before.
2017-01-05 15:54:37 	Initializing Study
2017-01-05 15:54:37 	Initializing Predictor configuration manager for Study
2017-01-05 15:54:37	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2017-01-05 15:54:37	Added ExtraTreesClassifier to default predictors
2017-01-05 15:54:37	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2017-01-05 15:54:37	Added ExtraTreesRegressor to default predictors
2017-01-05 15:54:37	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_b

Error loading species hg19 data: HTTP Error 404: Not Found

In [7]:
study.splicing.minimum_samples

10

In [8]:
not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

psi = study.splicing.singles.ix[not_outliers]
grouped = psi.groupby(study.sample_id_to_phenotype)
psi_filtered = grouped.apply(lambda x: x.dropna(axis=1, thresh=study.splicing.minimum_samples))

In [9]:
psi_filtered.head()

,isoform1=junction:chr10:101507148-101514285:+|isoform2=junction:chr10:101507148-101510125:+@exon:chr10:101510126-101510153:+@junction:chr10:101510154-101514285:+,isoform1=junction:chr10:102013297-102019974:-|isoform2=junction:chr10:102016234-102019974:-@exon:chr10:102016019-102016233:-@junction:chr10:102013297-102016018:-,isoform1=junction:chr10:102016234-102020722:-|isoform2=junction:chr10:102020077-102020722:-@exon:chr10:102019975-102020076:-@junction:chr10:102016234-102019974:-,isoform1=junction:chr10:102114390-102120490:+|isoform2=junction:chr10:102114390-102116288:+@exon:chr10:102116289-102116521:+@junction:chr10:102116522-102120490:+,isoform1=junction:chr10:102746733-102747069:-|isoform2=junction:chr10:102746954-102747069:-@exon:chr10:102746847-102746953:-@junction:chr10:102746733-102746846:-,isoform1=junction:chr10:103113986-103190101:+|isoform2=junction:chr10:103113986-103167197:+@novel_exon:chr10:103167198-103167290:+@junction:chr10:103167291-103190101:+,isoform1=junction:chr10:103348158-103360494:+|isoform2=junction:chr10:103348158-103354414:+@exon:chr10:103354415-103354495:+@junction:chr10:103354496-103360494:+,isoform1=junction:chr10:103361094-103368591:+|isoform2=junction:chr10:103361094-103364896:+@exon:chr10:103364897-103364969:+@junction:chr10:103364970-103368591:+,isoform1=junction:chr10:103361094-103369123:+|isoform2=junction:chr10:103361094-103368591:+@exon:chr10:103368592-103368694:+@junction:chr10:103368695-103369123:+,isoform1=junction:chr10:1034468-1041868:+|isoform2=junction:chr10:1034468-1038428:+@exon:chr10:1038429-1038599:+@junction:chr10:1038600-1041868:+,...,isoform1=junction:chrX:85233896-85282494:-|isoform2=junction:chrX:85236814-85282494:-@exon:chrX:85236741-85236813:-@junction:chrX:85233896-85236740:-,isoform1=junction:chrX:85236814-85302487:-|isoform2=junction:chrX:85282562-85302487:-@exon:chrX:85282495-85282561:-@junction:chrX:85236814-85282494:-,isoform1=junction:chrX:9693881-9707524:-|isoform2=junction:chrX:9694017-9707524:-@novel_exon:chrX:9693988-9694016:-@junction:chrX:9693881-9693987:-,isoform1=junction:chrX:99884984-99887481:-|isoform2=junction:chrX:99885864-99887481:-@exon:chrX:99885756-99885863:-@junction:chrX:99884984-99885755:-,isoform1=junction:chrX:99887566-99888927:-|isoform2=junction:chrX:99888537-99888927:-@exon:chrX:99888402-99888536:-@junction:chrX:99887566-99888401:-,isoform1=junction:chrX:99890744-99891604:-|isoform2=junction:chrX:99891205-99891604:-@novel_exon:chrX:99891188-99891204:-@junction:chrX:99890744-99891187:-,isoform1=junction:chrY:15026562-15026978:+|isoform2=junction:chrY:15026562-15026795:+@exon:chrY:15026796-15026894:+@junction:chrY:15026895-15026978:+,isoform1=junction:chrY:15028547-15029314:+|isoform2=junction:chrY:15028547-15028818:+@exon:chrY:15028819-15028972:+@junction:chrY:15028973-15029314:+,isoform1=junction:chrY:2709669-2712117:+|isoform2=junction:chrY:2709669-2710205:+@exon:chrY:2710206-2710283:+@junction:chrY:2710284-2712117:+,isoform1=junction:chrY:2710284-2713686:+|isoform2=junction:chrY:2710284-2712117:+@exon:chrY:2712118-2712298:+@junction:chrY:2712299-2713686:+
CVN_01,NaN,NaN,NaN,1.0,NaN,NaN,1.000000,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,1.0
CVN_02,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
CVN_03,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_04,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,1.0,NaN
CVN_05,NaN,NaN,NaN,1.0,NaN,NaN,0.481481,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,1.0,1.0


## Calculate *voyage* space via `bon voyage`

In [10]:
%%time

from bonvoyage import Waypoints

ws = Waypoints()

waypoints = psi_filtered.groupby(study.sample_id_to_phenotype).apply(
    lambda x: ws.fit_transform(x.dropna(how='all', axis=1)))

CPU times: user 3.51 s, sys: 4 ms, total: 3.51 s
Wall time: 3.5 s


In [11]:
waypoints.max()

0    1.0
1    1.0
dtype: float64

In [12]:
pd.DataFrame(ws.seed_data_transformed).max()

0    0.840892
1    0.999990
dtype: float64

In [13]:
transitions = study.phenotype_transitions + [('iPSC', 'MN')]
transitions

[(u'iPSC', u'NPC'), (u'NPC', u'MN'), ('iPSC', 'MN')]

In [14]:
from bonvoyage.voyages import Voyages

v = Voyages()

voyages = v.voyages(waypoints, transitions)
voyages.head()

,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition
0,isoform1=junction:chr10:102114390-102120490:+|...,0.000000,0.043478,0.043478,iPSC,NPC,$\nwarrow$,iPSC-NPC
1,isoform1=junction:chr10:102746733-102747069:-|...,0.012121,-0.012121,0.017142,iPSC,NPC,$\searrow$,iPSC-NPC
2,isoform1=junction:chr10:103348158-103360494:+|...,0.000000,-0.032258,0.032258,iPSC,NPC,$\swarrow$,iPSC-NPC
3,isoform1=junction:chr10:103361094-103368591:+|...,-0.007143,0.035714,0.036422,iPSC,NPC,$\nwarrow$,iPSC-NPC
4,isoform1=junction:chr10:103361094-103369123:+|...,0.043478,-0.043478,0.061488,iPSC,NPC,$\searrow$,iPSC-NPC


In [15]:
voyages['transition'] = voyages.group1 + '-' + voyages.group2
voyages.head()

,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition
0,isoform1=junction:chr10:102114390-102120490:+|...,0.000000,0.043478,0.043478,iPSC,NPC,$\nwarrow$,iPSC-NPC
1,isoform1=junction:chr10:102746733-102747069:-|...,0.012121,-0.012121,0.017142,iPSC,NPC,$\searrow$,iPSC-NPC
2,isoform1=junction:chr10:103348158-103360494:+|...,0.000000,-0.032258,0.032258,iPSC,NPC,$\swarrow$,iPSC-NPC
3,isoform1=junction:chr10:103361094-103368591:+|...,-0.007143,0.035714,0.036422,iPSC,NPC,$\nwarrow$,iPSC-NPC
4,isoform1=junction:chr10:103361094-103369123:+|...,0.043478,-0.043478,0.061488,iPSC,NPC,$\searrow$,iPSC-NPC


In [16]:
voyages.shape

(6384, 8)

### Add group1 and group2 modalities

In [17]:
study.supplemental.modalities_tidy.head()

,phenotype,event_id,modality
0,MN,isoform1=junction:chr10:102114390-102120490:+|...,included
1,MN,isoform1=junction:chr10:102746733-102747069:-|...,bimodal
2,MN,isoform1=junction:chr10:103113986-103190101:+|...,bimodal
3,MN,isoform1=junction:chr10:103348158-103360494:+|...,included
4,MN,isoform1=junction:chr10:103361094-103368591:+|...,excluded


In [18]:
groups = 'group1', 'group2'
voyages_modalities = voyages.copy()

for group in groups:
    voyages_modalities = voyages_modalities.merge(study.supplemental.modalities_tidy, 
                                       left_on=[group, 'event_id'], 
                                       right_on=['phenotype', 'event_id'], copy=False)
    print(voyages_modalities.shape)
    voyages_modalities = voyages_modalities.drop(['phenotype'], axis=1)
    voyages_modalities = voyages_modalities.rename(columns={'modality': '{}_modality'.format(group)})
print(voyages_modalities.shape)
voyages_modalities.head()

(6384, 10)
(6384, 11)
(6384, 10)


,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition,group1_modality,group2_modality
0,isoform1=junction:chr10:102114390-102120490:+|...,0.000000,0.043478,0.043478,iPSC,NPC,$\nwarrow$,iPSC-NPC,included,included
1,isoform1=junction:chr10:102114390-102120490:+|...,0.000000,0.043478,0.043478,iPSC,MN,$\nwarrow$,iPSC-MN,included,included
2,isoform1=junction:chr10:102114390-102120490:+|...,0.000000,0.000000,0.000000,NPC,MN,NaN,NPC-MN,included,included
3,isoform1=junction:chr10:102746733-102747069:-|...,0.012121,-0.012121,0.017142,iPSC,NPC,$\searrow$,iPSC-NPC,included,bimodal
4,isoform1=junction:chr10:102746733-102747069:-|...,0.109524,-0.109524,0.154890,iPSC,MN,$\searrow$,iPSC-MN,included,bimodal


## Add the data to the study and save it

In [19]:
study.supplemental.waypoints = waypoints
study.supplemental.voyages = voyages_modalities

In [20]:
study.save('singlecell_pnm_figure4_voyages', flotilla_dir='/projects/ps-yeolab/obotvinnik/flotilla_projects/')

Wrote datapackage to /projects/ps-yeolab/obotvinnik/flotilla_projects//singlecell_pnm_figure4_voyages/datapackage.json


In [21]:
# ll /projects/ps-yeolab/obotvinnik/flotilla_projects//singlecell_pnm_figure4_voyages

In [22]:
# ! md5sum /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure4_voyages/*csv